### LDSC heritability estimates and cohort-level inflation checks

#### Prepare input summary statistics files

In [72]:
cut -f2-9 rls_ukb_eur_sumstat_finalrs.tsv_final | head -2
cut -f2-9 rls_ukb_eur_sumstat_finalrs.tsv_final > rls_ukb_eur_sumstat_finalrs.tsv_final_tomunge
cut -f2-9 cartagene_eur_finalrs.tsv_finalannot > cartagene_eur_finalrs.tsv_final_tomunge
cut -f2-9 clsa_eur_finalrs.tsv_finalannot > clsa_eur_finalrs.tsv_finalannot_final_tomunge
cut -f2-9 allofus_eur_finalrs.tsv_final > allofus_eur_finalrs.tsv_final_tomunge
cut -f2-9 allofus_afr_finalrs.tsv > allofus_afr_finalrs.tsv_final_tomunge
cut -f2-9 allofus_aMr_finalrs.tsv > allofus_amr_finalrs.tsv_final_tomunge
cut -f2-9 AFR_GCST90477503_filt_BETASE_markername > AFR_GCST90477503_filt_BETASE_markername_tomunge
cut -f1-13 GCST90475828_filt_BETASE_pos_markeradd  > GCST90475828_filt_BETASE_pos_markeradd_tomunge 
cut -f1-13 EUR_GCST90475829_filt_filt_BETASE_markername > EUR_GCST90475829_filt_filt_BETASE_markername_tomunge

CHR	POS	EA	NEA	P	OR	STATUS	rsID
1	98749	C	T	0.22516	0.916979	9960099	rs377378992


#### Download previous EUR GWAS results

In [1]:
# wget http://ftp.ebi.ac.uk/pub/databases/gwas/summary_statistics/GCST90399001-GCST90400000/GCST90399568/harmonised/GCST90399568.h.tsv.gz
zcat GCST90399568.h.tsv.gz | awk 'BEGIN{OFS="\t"} NR==1 {print "CHR","POS","EA","NEA","P","BETA","rsID"} NR>1 {print $1,$2,$3,$4,$8,$5,$9}'  > GCST90399568_formatted.tsv

In [2]:
module load ldsc/1.0.1-20200724

[+] Loading singularity  4.2.2  on cn3141 
[+] Loading ldsc, version 1.0.1-20200724...


#### Download and prepare UKB reference files

In [ ]:
#ls UKBB.ALL.ldscore.tar.gz
tar -xvzf UKBB.ALL.ldscore.tar.gz
mkdir afr_ldscores
cp UKBB.ALL.ldscore/UKBB.AFR.8LDMS.rsid.l2.ldscore.gz afr_ldscores/
cp UKBB.ALL.ldscore/UKBB.AFR.8LDMS.rsid.l2.annot.gz afr_ldscores/
zcat UKBB.ALL.ldscore/UKBB.AFR.8LDMS.rsid.l2.ldscore.gz | \
awk 'BEGIN {OFS="\t"} NR==1 {print $1, $2, $3, "L2"} NR>1 {print $1, $2, $3, $4}' | \
gzip > UKBB.ALL.ldscore/UKBB.AFR.8LDMS.rsid.l2.weight.l2.ldscore.gz

zcat UKBB.ALL.ldscore/UKBB.AFR.rsid.l2.ldscore.gz | awk 'NR > 1 {print $2}' > afr_rsid_list.txt
zcat munged_MVA_afr_only_finalrs.tsv_final.sumstats.gz | awk '{print $1"\t"$2"\t"$3}' | grep -Ff afr_rsid_list.txt > tmp.txt
(echo -e "SNP\tA1\tA2"; cat tmp.txt) > afr_merge_alleles.txt
awk 'NF == 3 && $2 ~ /^[ACGT]$/ && $3 ~ /^[ACGT]$/' afr_merge_alleles.txt > tmp_clean.txt
(echo -e "SNP\tA1\tA2"; cat tmp_clean.txt) > afr_merge_alleles_clean.txt
head -3 afr_merge_alleles_clean.txt
wc -l afr_merge_alleles_clean.txt

zcat UKBB.ALL.ldscore/UKBB.AMR.rsid.l2.ldscore.gz | awk 'NR > 1 {print $2}' > amr_rsid_list.txt
zcat munged_MVA_amr_only_finalrs.tsv_final.sumstats.gz | awk '{print $1"\t"$2"\t"$3}' | grep -Ff amr_rsid_list.txt > tmp2.txt
(echo -e "SNP\tA1\tA2"; cat tmp2.txt) > amr_merge_alleles.txt
awk 'NF == 3 && $2 ~ /^[ACGT]$/ && $3 ~ /^[ACGT]$/' amr_merge_alleles.txt > tmp2_clean.txt
(echo -e "SNP\tA1\tA2"; cat tmp2_clean.txt) > amr_merge_alleles_clean.txt
head -3 amr_merge_alleles_clean.txt
wc -l amr_merge_alleles_clean.txt

#### Prepare and munge each GWAS summary statistic

In [48]:
#EUR_GWAS included
munge_sumstats.py --sumstats rls_ukb_eur_sumstat_finalrs.tsv_final_tomunge --N-cas 7317 --N-con 106479 \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_rls_ukb_eur_sumstat_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats allofus_eur_finalrs.tsv_final_tomunge --N-cas 8300 --N-con 86961 \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_allofus_eur_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats EUR_GCST90475829_filt_filt_BETASE_markername_tomunge --N-cas 15475 --N-con 428445 --ignore OR,MARKERNAME \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_MVA_eur_only_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats clsa_eur_finalrs.tsv_finalannot_final_tomunge --N-cas 4980 --N-con 15990 \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_clsa_eur_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats cartagene_eur_finalrs.tsv_final_tomunge --N-cas 921 --N-con 1307 \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_cartagene_eur_finalrs.tsv_final --chunksize 500000

#EUR_2024_RLS_GWAS (Schormair et al)
munge_sumstats.py --sumstats GCST90399568_formatted.tsv --N-cas 7248 --N-con 19802 \
--snp rsID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_2024_EUR_RLS_final --chunksize 500000 

#AFR_GWAS
munge_sumstats.py --sumstats allofus_afr_finalrs.tsv_final_tomunge --N-cas 873 --N-con 33876  \
--snp rsID --a1 EA --a2 NEA --merge-alleles afr_merge_alleles_clean.txt --out munged_allofus_afr_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats MVA_afr_only_finalrs.tsv_finalannot_tomunge --N-cas 1303 --N-con 119437 \
--snp rsID --a1 EA --a2 NEA --merge-alleles afr_merge_alleles_clean.txt --out munged_MVA_afr_only_finalrs.tsv_final --chunksize 500000 

#AMR_GWAS
munge_sumstats.py --sumstats allofus_amr_finalrs.tsv_final_tomunge --N-cas 876 --N-con 33909 \
--snp rsID --a1 EA --a2 NEA --merge-alleles amr_merge_alleles_clean.txt --out munged_allofus_amr_finalrs.tsv_final --chunksize 500000 
munge_sumstats.py --sumstats GCST90475828_filt_BETASE_pos_markeradd_tomunge  --N-cas 1148 --N-con 57993 --ignore OR,MARKERNAME \
--snp rsID --a1 EA --a2 NEA --merge-alleles amr_merge_alleles_clean.txt --out munged_MVA_amr_only_finalrs.tsv_final --chunksize 500000

*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./munge_sumstats.py \
--out munged_rls_ukb_eur_sumstat_finalrs.tsv_final \
--merge-alleles w_hm3.snplist \
--N-con 106479.0 \
--chunksize 500000 \
--N-cas 7317.0 \
--a1 EA \
--a2 NEA \
--snp rsID \
--sumstats rls_ukb_eur_sumstat_finalrs.tsv_final_tomunge 

Interpreting column names as follows:
P:	p-Value
rsID:	Variant ID (e.g., rs number)
EA:	Allele 1, interpreted as ref allele for signed sumstat.
OR:	Odds ratio (1 --> no effect; above 1 --> A1 is risk increasing)
NEA:	Allele 2, interpreted as non-ref allele for signed sumstat.

Reading list of SNPs for allele merge from w_hm3.snplist
Read 1217311 SNPs for allele merge.
Reading sumstats fro

In [13]:
ls -lh munged*gz 

-rw-r----- 1 akcimenf2 CARD_AUX 8.1M Aug 20 21:42 munged_MVA_afr_only_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.1M Aug 20 21:48 munged_MVA_amr_only_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.4M Aug 20 21:20 munged_MVA_eur_only_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 6.3M Aug  7 17:34 munged_afrmeta_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.1M Aug 20 21:38 munged_allofus_afr_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 7.7M Aug 20 21:44 munged_allofus_amr_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.4M Aug 20 21:28 munged_allofus_eur_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 7.9M Aug  7 00:48 munged_amrmeta_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.3M Aug 20 21:26 munged_cartagene_eur_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akcimenf2 CARD_AUX 8.4M Aug 20 21:24 munged_clsa_eur_finalrs.tsv_final.sumstats.gz
-rw-r----- 1 akc

#### Heritability analysis across EUR cohorts

In [53]:
ldsc.py --h2 munged_rls_ukb_eur_sumstat_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ --w-ld-chr eur_w_ld_chr/ --out ukb_current --samp-prev 0.0643 --pop-prev 0.0644
ldsc.py --h2 munged_allofus_eur_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ --w-ld-chr eur_w_ld_chr/ --out munged_allofus_eur --samp-prev 0.087 --pop-prev 0.05
ldsc.py --h2 munged_MVA_eur_only_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ --w-ld-chr eur_w_ld_chr/ --out munged_MVA_eur --samp-prev 0.035 --pop-prev 0.05
ldsc.py --h2 munged_clsa_eur_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ --w-ld-chr eur_w_ld_chr/ --out munged_clsa_eur --samp-prev 0.24 --pop-prev 0.06
ldsc.py --h2 munged_cartagene_eur_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ --w-ld-chr eur_w_ld_chr/ --out munged_cartagene_eur --samp-prev 0.41 --pop-prev 0.06

*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--h2 munged_MVA_eur_only_finalrs.tsv_final.sumstats.gz \
--ref-ld-chr eur_w_ld_chr/ \
--out munged_MVA_eur \
--pop-prev 0.06 \
--samp-prev 0.035 \
--w-ld-chr eur_w_ld_chr/ 

Beginning analysis at Wed Aug 20 23:53:21 2025
Reading summary statistics from munged_MVA_eur_only_finalrs.tsv_final.sumstats.gz ...
Read summary statistics for 1185653 SNPs.
Reading reference panel LD Score from eur_w_ld_chr/[1-22] ... (ldscore_fromlist)
Read reference panel LD Scores for 1290028 SNPs.
Removing partitioned LD Scores with zero variance.
Reading regression weight LD Score from eur_w_ld_chr/[1-22] ... (ldscore_fromlist)
Read regression weight 

In [26]:
#Ancestry-stratified meta-analyses
zcat AUG2025_AFR_META1.tbl.gz | wc -l
zcat AUG2025_AMR_META1.tbl.gz | wc -l
zcat AUG2025_EUR_META1.tbl.gz | wc -l

18832804
12403012
9350470


In [29]:
wc -l AUG2025_AFR_META1_RS_anno
wc -l AUG2025_AMR_META1_RS_anno
wc -l AUG2025_EUR_META1_RS_anno
head -2 AUG2025_*R_META1_RS_anno

18832804 AUG2025_AFR_META1_RS_anno
12403012 AUG2025_AMR_META1_RS_anno
9350466 AUG2025_EUR_META1_RS_anno
==> AUG2025_AFR_META1_RS_anno <==
SNPID	CHR	POS	EA	NEA	BETA	SE	P	STATUS	rsID
chr11:12541586:A:G	11	12541586	A	G	0.0239	0.0334	0.4751	9960099	rs1426593

==> AUG2025_AMR_META1_RS_anno <==
SNPID	CHR	POS	EA	NEA	BETA	SE	P	STATUS	rsID
chr11:12541586:A:G	11	12541586	A	G	0.0017	0.0333	0.9598	9960099	rs1426593

==> AUG2025_EUR_META1_RS_anno <==
SNPID	CHR	POS	EA	NEA	BETA	SE	P	STATUS	rsID
chr11:12541586:A:G	11	12541586	A	G	-0.0076	0.0084	0.3643	9960099	rs1426593


### Munge meta-analyses (λ and λ1000)

In [39]:
munge_sumstats.py --sumstats AUG2025_AFR_META1_RS_anno --N-cas 2176 --N-con 153313 --maf-min 0.01 --snp rsID \
--ignore SNPID --a1 EA --a2 NEA --merge-alleles afr_merge_alleles_clean.txt --out munged_afrmeta_finalrs.tsv_final --chunksize 500000

munge_sumstats.py --sumstats AUG2025_AMR_META1_RS_anno --N-cas 2024 --N-con 91902 --maf-min 0.01 --snp rsID \
--ignore SNPID --a1 EA --a2 NEA --merge-alleles amr_merge_alleles_clean.txt --out munged_amrmeta_finalrs.tsv_final --chunksize 500000 

munge_sumstats.py --sumstats AUG2025_EUR_META1_RS_anno --N-cas 36993 --N-con 639182 --maf-min 0.01 --snp rsID \
--ignore SNPID --a1 EA --a2 NEA --merge-alleles w_hm3.snplist --out munged_eurmeta_finalrs.tsv_final --chunksize 500000 

### Heritability analysis across AFR, AMR, and EUR meta-analyses

In [58]:
ldsc.py --h2 munged_afrmeta_finalrs.tsv_final.sumstats.gz --ref-ld UKBB.ALL.ldscore/UKBB.AFR.rsid   \
--w-ld UKBB.ALL.ldscore/UKBB.AFR.rsid --out munged_meta_afr  --print-coefficients --samp-prev 0.014 --pop-prev 0.009

ldsc.py --h2 munged_amrmeta_finalrs.tsv_final.sumstats.gz --ref-ld UKBB.ALL.ldscore/UKBB.AMR.rsid   \
--w-ld UKBB.ALL.ldscore/UKBB.AMR.rsid --out munged_meta_amr  --print-coefficients --samp-prev 0.022 --pop-prev 0.009

ldsc.py --h2 munged_eurmeta_finalrs.tsv_final.sumstats.gz --ref-ld-chr eur_w_ld_chr/   \
#--w-ld-chr eur_w_ld_chr/ --out munged_meta_eur  --print-coefficients --samp-prev 0.055 --pop-prev 0.05